In [2]:
from langchain_groq import ChatGroq
import chromadb
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [15]:
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    groq_api_key='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'
)

loader = WebBaseLoader("https://jobs.apple.com/en-ca/details/200578873/computer-vision-and-deep-learning-engineer?team=MLAI")
# loader = WebBaseLoader("https://www.linkedin.com/jobs/view/4023109830")
page_data=loader.load().pop().page_content
# print(page_data)

In [16]:
from langchain_core.prompts import PromptTemplate

prompt_extract=PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
     {page_data}
    ### INSTRUCTION:
    The scraped  text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing
    following keys: 'role','experience','skills' and 'description'.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE):
    """
)

chain_extract = prompt_extract | llm
res=chain_extract.invoke(input={'page_data':page_data})
print(type(res.content))

<class 'str'>


In [17]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res=json_parser.parse(res.content)
json_res

{'role': 'Computer Vision and Deep Learning Engineer',
 'experience': "PhD or Master's degree in Computer Science, Electrical Engineering, or related field",
 'skills': ['Strong foundation in computer vision, machine learning, and deep learning',
  'Knowledge in the field of 3D computer vision techniques (e.g. stereo, structure from motion, camera model)',
  'Proficiency in Python and deep learning frameworks (e.g. PyTorch)',
  'Experience in working with large scale datasets',
  'Experience with optimizing models in both size and runtime'],
 'description': 'We are seeking a talented Computer Vision Research Engineer to join our team. The job will focus on developing deep learning models for 3D reconstruction and perception using state-of-the-art deep learning techniques. As a key member of our team, you’ll be involved in the entire process of designing and training these models whose derivatives will be deployed in various Apple products.'}

In [18]:
import pandas as pd

df=pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [19]:
import chromadb
import uuid
client = chromadb.PersistentClient('vectorstore')
collection= client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _,row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links":row["Links"]},
                       ids=[str(uuid.uuid4())])


In [20]:
job = json_res
job['skills']


['Strong foundation in computer vision, machine learning, and deep learning',
 'Knowledge in the field of 3D computer vision techniques (e.g. stereo, structure from motion, camera model)',
 'Proficiency in Python and deep learning frameworks (e.g. PyTorch)',
 'Experience in working with large scale datasets',
 'Experience with optimizing models in both size and runtime']

In [21]:
links = collection.query(query_texts=job["skills"], n_results=2).get('metadatas',[])

In [22]:
prompt_email = PromptTemplate.from_template(
    '''
    ### JOB DESCRIPTION:
    {job_description}
     ### INSTRUCTION:
    you are Rajkumar Arumugam Jeeva.
    I'm a passionate computer vision engineer pursuing my Master's in Applied Science in Biomedical Engineering at the
    University of Ottawa. I have two years of hands-on experience in computer vision, working on tasks like object detection,
    segmentation, and building machine learning models for classification. I’ve also worked with machine vision cameras and
    edge devices like Raspberry Pi, Pico, Arduino, and ESP modules. My research focuses on automatic video quality
    assessment for gait videos using pose estimation. I have a publication in IEEE on applying optical flow to laryngeal videos
    to detect abnormalities.
    your job is to write a cold email to the Hiring manager regarding the job mentioned describing the I'm the 
    suitable candidate for the  job.
    Also add the most relevant projects from the following links to showcase Rajkumar's  portfolio :{link_list}
    Remember, you are Rajkumar Arumugam Jeeva.
    Do not provise a preamble.
    ### EMAIL (NO PREAMBLE):   '''
)

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Application for Computer Vision and Deep Learning Engineer Role

Dear Hiring Manager,

I am writing to express my interest in the Computer Vision and Deep Learning Engineer position at Apple. As a passionate computer vision engineer with a strong foundation in machine learning and deep learning, I am confident that I would be an excellent fit for this role.

With two years of hands-on experience in computer vision, I have developed expertise in object detection, segmentation, and building machine learning models for classification. My research focus on automatic video quality assessment for gait videos using pose estimation has given me a unique understanding of the importance of accurate and efficient computer vision models. Additionally, my publication in IEEE on applying optical flow to laryngeal videos to detect abnormalities demonstrates my ability to apply computer vision techniques to real-world problems.

I am particularly drawn to this role because of the opportunity 